In [124]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange

In [125]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) > 1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)

In [126]:
def followingInCommon(node1,node2):  
    list1 = dictionary[node1]
    list2 = dictionary[node2]  
    common_elements = set(list1).intersection(list2) 
    return len(common_elements)
    
# Test    
followingInCommon(3849054,161276) 

1137

In [127]:
def indirectFollowings(node_source,node_sink):
    count = 0
    list1 = dictionary[node_source]
    for node in list1:
        if node_sink in dictionary[node]:
            count += 1
    return count
    
# Test    
indirectFollowings(3849054,161276) 

67

In [43]:
indirectFollowings(161276,3849054) 

66

In [145]:
#real_edges1 = pd.read_csv("data_generated/real_edges1.csv", sep='\t')
#real_edges2 = pd.read_csv("data_generated/real_edges2.csv", sep='\t')
fake_edges3 = pd.read_csv("data_generated/fake_edges3.csv", sep='\t')
real_edges3 = pd.read_csv("data_generated/real_edges3.csv", sep='\t')

#fake_edges2 = pd.read_csv("data_generated/fake_edges2.csv", sep='\t')
test_data = pd.read_csv("data_generated/test_data.csv", sep='\t')

In [146]:
#real_edges1['Real'] = 1
#real_edges2['Real'] = 1
real_edges3['Real'] = 1
fake_edges3['Real'] = 0

In [147]:
real_edges3['Common_Followings'] = real_edges3.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
print('a')
real_edges3['Com_Followings_Ratio'] = real_edges3['Common_Followings']/real_edges3['Source_Followings']
print('b')
real_edges3['Followers_Ratio'] = real_edges3['Sink_Followers']/real_edges3['Source_Followers']
print('c')
real_edges3['Indirect_Followings'] = real_edges3.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
print('d')
real_edges3['Inv_Indirect_Followings'] = real_edges3.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)

a
b
c
d


In [149]:
fake_edges3['Common_Followings'] = fake_edges3.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
print('a')
fake_edges3['Com_Followings_Ratio'] = fake_edges3['Common_Followings']/fake_edges3['Source_Followings']
print('b')
fake_edges3['Followers_Ratio'] = fake_edges3['Sink_Followers']/fake_edges3['Source_Followers']
print('c')
fake_edges3['Indirect_Followings'] = fake_edges3.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
print('d')
fake_edges3['Inv_Indirect_Followings'] = fake_edges3.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)

a
b
c
d


In [150]:
test_data['Common_Followings'] = test_data.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
print('a')
test_data['Com_Followings_Ratio'] = test_data['Common_Followings']/test_data['Source_Followings']
print('b')
test_data['Followers_Ratio'] = test_data['Sink_Followers']/test_data['Source_Followers']
print('c')
test_data['Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
print('d')
test_data['Inv_Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)

a
b
c
d


In [151]:
test_data.tail(10)

,Source,Sink,Source_Followings,Source_Followers,Sink_Followers,Distance,Common_Followings,Com_Followings_Ratio,Followers_Ratio,Indirect_Followings,Inv_Indirect_Followings
1990,1809767,335012,86,8,34,3.0,0,0.000000,4.250000,0,0
1991,4345222,1766631,887,204,87,2.0,0,0.000000,0.426471,1,0
1992,4518291,2534067,223,2,24,3.0,0,0.000000,12.000000,0,0
1993,4055860,1140987,1999,24,821,2.0,0,0.000000,34.208333,15,0
1994,465530,3426792,335,16,161,2.0,9,0.026866,10.062500,4,1
1995,1461386,2341683,53,16,2,3.0,0,0.000000,0.125000,0,0
1996,4057755,1871227,95,53,41,3.0,0,0.000000,0.773585,0,0
1997,4242514,1413468,27,6,2,3.0,0,0.000000,0.333333,0,0
1998,555531,1290080,56,7,3,3.0,0,0.000000,0.428571,0,0
1999,1707829,2373045,244,10,2,3.0,0,0.000000,0.200000,0,0


In [152]:
#real_edges1.to_csv("data_generated/real_edges1.csv", sep='\t', index=False)
#real_edges2.to_csv("data_generated/real_edges2.csv", sep='\t', index=False)
real_edges3.to_csv("data_generated/real_edges3.csv", sep='\t', index=False)
fake_edges3.to_csv("data_generated/fake_edges3.csv", sep='\t', index=False)
test_data.to_csv("data_generated/test_data.csv", sep='\t', index=False)

In [154]:
#fake_edges2['Real'] = 0
frames = [real_edges3,fake_edges3]
dataset = pd.concat(frames, ignore_index=True)

# Saving real edges
dataset.to_csv("data_models/dataset5.csv", sep='\t', index=False)

In [69]:
def checkSink(node):
    check = 0
    if ( (data['Source'] == node).any() ):
        check = 1         
    return check

In [70]:
test_data2 = test_data

In [71]:
test_data2['SINK_SOURCE'] = test_data2.apply(lambda x: checkSink(x['Sink']), axis=1)

In [73]:
test_data.to_csv("data_generated/test_data2.csv", sep=',', index=False)

In [98]:
data.head(10)


,Source,Source_Followings
0,540762,143
1,2129843,21
2,3361377,764195
3,1199298,297
4,1392121,3808
5,4762204,4519
6,1840280,333
7,2673351,227
8,2672578,258
9,400117,1225
